Définir le chemin pour accéder à tous les fichiers des matchs

In [ ]:

import pandas as pd
import json
import os
from pathlib import Path

Data_path = Path('../data/ligue1')

print(len(list(Data_path.glob('*.json'))))


267


Identification des équipes et compter les buts

Permet de récupérer les résultats du match

In [2]:
def resultat_match(fichier_path):
    with open(fichier_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)

    match_id = Path(fichier_path).stem

    equipes = df[df['type.name'] == 'Starting XI']['team.name'].tolist()

    equipe_domicile = equipes[0]
    equipe_exterieur = equipes[1]

    tirs = df[df['type.name'] == 'Shot']
    buts = tirs[tirs['shot.outcome.name'] == 'Goal']

    buts_domicile = len(buts[buts['team.name'] == equipe_domicile])
    buts_exterieur = len(buts[buts['team.name'] == equipe_exterieur])

    return {'match ID' : match_id, 'equipe_domicile': equipe_domicile, 'equipe_exterieur': equipe_exterieur, 'buts_domicile': buts_domicile, 'buts_exterieur': buts_exterieur}

test_match = Data_path / '3900512.json'
resultat = resultat_match(test_match)

print(f"{resultat['equipe_domicile']} {resultat['buts_domicile']} - {resultat['buts_exterieur']} {resultat['equipe_exterieur']}")

Montpellier 3 - 1 Stade de Reims


Charger tous les matchs 

In [3]:
def charger_tous_les_matchs(dossier_path):
    dossier = Path(dossier_path)
    fichiers = list(dossier.glob('*.json'))
    
    resultats = []
    erreurs = []

    for i, fichier in enumerate(fichiers):
        if 'checkpoint' in str(fichier):
            continue
        try:
            resultat = resultat_match(fichier)
            resultats.append(resultat)
        except Exception as e:
            erreurs.append({'fichier': fichier.name, 'erreur': str(e)})

    print(f'{len(resultats)} matchs chargés, {len(erreurs)} erreurs')

    if erreurs:
        print(f"Fichiers avec erreurs : {[e['fichier'] for e in erreurs]}")

    return pd.DataFrame(resultats)

df_matchs = charger_tous_les_matchs(Data_path)

df_matchs.head(10)

267 matchs chargés, 0 erreurs


,match ID,equipe_domicile,equipe_exterieur,buts_domicile,buts_exterieur
0,3900490,Rennes,Paris Saint-Germain,0,1
1,3900491,Saint-Étienne,Stade de Reims,2,0
2,3900492,Guingamp,Lorient,2,2
3,3900493,Troyes,Lyon,0,1
4,3900494,Gazélec Ajaccio,Bordeaux,2,0
5,3900495,Bastia,Caen,1,0
6,3900496,Toulouse,Montpellier,1,1
7,3900497,OGC Nice,Lille,0,0
8,3900498,AS Monaco,Angers,1,0
9,3900499,Nantes,Marseille,0,1


Calculer le classement

In [13]:
def calcul_classement(df_matchs):
    stats = {}

    for i, match in df_matchs.iterrows():
        equipe_dom = match['equipe_domicile']
        equipe_ext = match['equipe_exterieur']
        buts_dom = match['buts_domicile']
        buts_ext = match['buts_exterieur']

        for equipe in [equipe_dom, equipe_ext]:
            if equipe not in stats:
                stats[equipe] = {
                    'Equipe': equipe,
                    'Pts' : 0,
                    'J': 0,
                    'V': 0,
                    'N': 0,
                    'D': 0,
                    'BP': 0,
                    'BC': 0
                }
        stats[equipe_dom]['J'] += 1
        stats[equipe_ext]['J'] += 1

        stats[equipe_dom]['BP'] += buts_dom
        stats[equipe_dom]['BC'] += buts_ext
        stats[equipe_ext]['BP'] += buts_ext
        stats[equipe_ext]['BC'] += buts_dom

        if buts_dom > buts_ext:
            stats[equipe_dom]['Pts'] += 3
            stats[equipe_dom]['V'] += 1
            stats[equipe_ext]['D'] += 1
        elif buts_dom < buts_ext:
            stats[equipe_ext]['Pts'] += 3
            stats[equipe_ext]['V'] += 1
            stats[equipe_dom]['D'] += 1
        else:
            stats[equipe_dom]['Pts'] += 1
            stats[equipe_ext]['Pts'] += 1
            stats[equipe_dom]['N'] += 1
            stats[equipe_ext]['N'] += 1

    df_classement = pd.DataFrame(stats.values())

    df_classement['Diff'] = df_classement['BP'] - df_classement['BC']
    
    df_classement = df_classement.sort_values(by=['Pts', 'Diff', 'BP'], ascending = False)

    df_classement = df_classement.reset_index(drop=True)
    df_classement.index = df_classement.index + 1

    return df_classement

df_classement = calcul_classement(df_matchs)
df_classement 



,Equipe,Pts,J,V,N,D,BP,BC,Diff
1,Paris Saint-Germain,64,26,20,4,2,74,13,61
2,Lille,49,27,13,10,4,32,20,12
3,AS Monaco,47,27,13,8,6,39,35,4
4,Lyon,46,27,14,4,9,51,34,17
5,OGC Nice,46,27,13,7,7,31,22,9
6,Montpellier,41,27,12,5,10,39,33,6
7,Marseille,41,27,9,14,4,31,26,5
8,Saint-Étienne,39,26,11,6,9,27,21,6
9,Bastia,36,26,9,9,8,21,21,0
10,Caen,35,27,10,5,12,26,38,-12
